# Imports

In [ ]:
# Planetary computer Search
from pystac_client import Client
import planetary_computer as pc

# DataFrames - Read and Write
import pandas as pd
import numpy as np
import geopandas as gpd

# Raster Operations
import rioxarray as rxr
from xrspatial.zonal import stats

# Raster Visualization (xArray can be used but computation heavy) 
import rasterio
from rasterio import windows
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Load Boundary data

In [ ]:
def check_projection(geo_df):
    current_crs = geo_df.crs 
    if current_crs == 4326 :
        return geo_df 
    else : 
        geo_df = geo_df.to_crs("4326")
        return geo_df

In [ ]:
delhi_subdistricts = gpd.read_file('delhi.gpkg')
delhi_subdistricts = check_projection(delhi_subdistricts)

In [ ]:
# set this as defalt CRS, this CRS will be used for all data
def_crs = delhi_subdistricts.crs
def_crs

In [ ]:
# get bounding box around delhi region. 
# This values will be ised to query the Planetary computer data catalog
delhi = delhi_subdistricts.dissolve()
bounds_df = delhi.bounds.squeeze()
bounds = [bounds_df['minx'], bounds_df['miny'], bounds_df['maxx'], bounds_df['maxy']]
bounds

# Query for Landcover (raster)

In [ ]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(collections=["esa-worldcover"], bbox=bounds)

items = list(search.get_items())
print(f"Returned {len(items)} Items")

In [ ]:
asset_href = items[0].assets["map"].href
print(asset_href)

In [ ]:
signed_href = pc.sign(asset_href)

# Landcover visualization 

In [ ]:
with rasterio.open(signed_href) as src:
    aoi_window = windows.from_bounds(transform=src.transform, *bounds)
    raster_data = src.read(1, window=aoi_window)

fig, ax = plt.subplots(figsize=(12, 12))

ax.set_axis_off()
ax.imshow(raster_data);

In [ ]:
with rasterio.open(signed_href) as src:
    colormap_def = src.colormap(1)
    colormap = [np.array(colormap_def[i]) / 255 for i in range(len(colormap_def))]

cmap = ListedColormap(colormap)
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_axis_off()
ax.imshow(raster_data, cmap=cmap, vmin=0,  vmax=(len(colormap_def) - 1),  interpolation="nearest");

# Converting to xArray DataFrame for computation

In [ ]:
delhi_landcover = rxr.open_rasterio(signed_href)
delhi_landcover

In [ ]:
delhi_landcover_clipped = delhi_landcover.rio.clip(delhi.geometry.values, def_crs)

In [ ]:
delhi_landcover_clipped.rio.to_raster("delhi_landcover_clipped.tif")

In [ ]:
concatList = []
for _, sub_district in delhi_subdistricts.iterrows():
    shape_geom = sub_district['geometry']
    raster = delhi_landcover_clipped.rio.clip(shape_geom, def_crs)
    stats_df = stats(zones=raster[0], values=raster[0])
    stats_df['area_km'] = stats_df['count']*10*10/1000000
    stats_df = stats_df[['zone', 'area_km']].set_index('zone').drop(0).transpose()
    stats_df['sub_district'] = sub_district['NAME']
    concatList.append(stats_df)
area_df = pd.concat(concatList)
area_df = area_df.set_index('sub_district')

In [ ]:
calss_values = {
  10: "Trees",
  20: "Shrubland",
  30: "Grassland",
  40: "Cropland",
  50: "Built-up",
  60: "Barren / sparse vegetation",
  70: "Snow and ice",
  80: "Open water",
  90: "Herbaceous wetland",
  95: "Mangroves",
  100: "Moss and lichen"
}

In [ ]:
area_df = area_df.rename(columns=calss_values)
area_df.head()

# Export the csv file with area value for each category

In [ ]:
area_df.to_csv('delhi_landcover_area_sqkm.csv')